## Michael Owen GAN Project

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, LeakyReLU, Reshape, AveragePooling2D
from tensorflow.keras.layers import ZeroPadding2D, Flatten, Dense, Input, Activation,MaxPool2D, BatchNormalization, Dropout
import tensorflow_docs.vis.embed as embed
import glob, os, imageio
mnist = tf.keras.datasets.fashion_mnist

## Tensorflow Image generation via two competing networks (generator and discriminator) 

In [2]:
def get_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer="he_uniform", input_shape=(28,28,1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer="he_uniform"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (4,4), strides=(1,1), padding='same', kernel_initializer="he_uniform"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid', kernel_initializer="glorot_uniform"))
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
 
def get_generator(lat_space):
    model = Sequential()
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, kernel_initializer="he_uniform",activation='relu', input_dim=lat_space))
    #model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', activation='relu', kernel_initializer="he_uniform"))
    #model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='relu', kernel_initializer="he_uniform"))
    #model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(1,1), padding='same', activation='relu', kernel_initializer="he_uniform"))
    #model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer="glorot_uniform"))
    return model

def get_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def get_data():
    (trainX, trainy), (_, _) = mnist.load_data()
    X_t = np.expand_dims(trainX, axis=3)
    X_t = X_t.astype('float32')
    # [0,255] -> [-1,1]
    X_t = (X_t - 127.5) / 127.5
    return X_t

def get_real_imgs(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X_r = dataset[ix]
    y = np.ones((n_samples, 1))
    return X_r, y

 
def get_fake_imgs(generator, lat_space, n_samples):
    x_input = np.random.normal(0,1,size=[n_samples,lat_space])
    X_f = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X_f, y

def save_gen_imgs(step, g_model, lat_space, n_samples=200):
    plt.figure(figsize=(15,15))
    X, _ = get_fake_imgs(g_model, lat_space, n_samples)
    X = (X + 1) / 2.0
    X = X.reshape(n_samples,28,28)
    for i in range(X.shape[0]):
        plt.subplot(10, 20, 1 + i)
        plt.axis('off')
        plt.imshow(X[i], cmap='gray')
    plt.savefig('results_new9/generated_plot_%03d.png' % (step+1))
    plt.close()
    
def loss_acc_plot(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
    plt.subplot(2, 1, 1)
    plt.plot(d1_hist, label='d-real')
    plt.plot(d2_hist, label='d-fake')
    plt.plot(g_hist, label='gen')
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(a1_hist, label='acc-real')
    plt.plot(a2_hist, label='acc-fake')
    plt.legend()
    plt.savefig('results_new9/plot_line_plot_loss.png')
    plt.close()

In [3]:
def train_gan():
    n_epochs=200
    lat_space = 150
    discriminator = get_discriminator()
    generator = get_generator(lat_space)
    gan_combined = get_gan(generator, discriminator)
    data = get_data()
    n_batch = 256
    bat_per_epo = int(data.shape[0] / n_batch)
    iterations = bat_per_epo * n_epochs
    half_batch = int(n_batch / 2)
    d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
    for i in range(iterations):
        ind = np.random.randint(0,data.shape[0],half_batch)
        X_real, y_real = get_real_imgs(data[ind], half_batch)
        
        d_loss1, d_acc1 = discriminator.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = get_fake_imgs(generator, lat_space, half_batch)
        
        d_loss2, d_acc2 = discriminator.train_on_batch(X_fake, y_fake)
        
        X_gan_combined = np.random.normal(0,1,size=[n_batch,lat_space])
        
        y_gan_combined = np.ones((n_batch, 1))
        
        g_loss = gan_combined.train_on_batch(X_gan_combined, y_gan_combined)
        # print batch performance
        if (i+1) % 300 == 0 or (i+1) == 100:
            print('Step%d, d_r_loss=%.3f, d_f_loss=%.3f g_loss=%.3f, d_r_acc=%d, d_f_acc=%d' %(i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
        # record history
        d1_hist.append(d_loss1)
        d2_hist.append(d_loss2)
        g_hist.append(g_loss)
        a1_hist.append(d_acc1)
        a2_hist.append(d_acc2)
        if (i+1) % 400 == 0 or (i+1) == 1 or (i+1) == 10 or (i+1) == 100 or (i+1) == 150 or (i+1) >=46790:
            save_gen_imgs(i, generator, lat_space)
    loss_acc_plot(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

In [4]:
train_gan()

Step100, d_r_loss=0.233, d_f_loss=0.810 g_loss=0.683, d_r_acc=88, d_f_acc=4
Step300, d_r_loss=0.466, d_f_loss=0.348 g_loss=1.982, d_r_acc=77, d_f_acc=88
Step600, d_r_loss=0.377, d_f_loss=0.349 g_loss=2.296, d_r_acc=81, d_f_acc=89
Step900, d_r_loss=0.376, d_f_loss=0.399 g_loss=1.830, d_r_acc=85, d_f_acc=92
Step1200, d_r_loss=0.541, d_f_loss=0.416 g_loss=1.535, d_r_acc=70, d_f_acc=87
Step1500, d_r_loss=0.680, d_f_loss=0.493 g_loss=1.370, d_r_acc=59, d_f_acc=80
Step1800, d_r_loss=0.532, d_f_loss=0.523 g_loss=1.293, d_r_acc=64, d_f_acc=80
Step2100, d_r_loss=0.541, d_f_loss=0.512 g_loss=1.321, d_r_acc=68, d_f_acc=77
Step2400, d_r_loss=0.562, d_f_loss=0.524 g_loss=1.194, d_r_acc=75, d_f_acc=78
Step2700, d_r_loss=0.665, d_f_loss=0.602 g_loss=1.130, d_r_acc=60, d_f_acc=71
Step3000, d_r_loss=0.734, d_f_loss=0.599 g_loss=1.321, d_r_acc=56, d_f_acc=67
Step3300, d_r_loss=0.619, d_f_loss=0.508 g_loss=1.178, d_r_acc=64, d_f_acc=80
Step3600, d_r_loss=0.625, d_f_loss=0.562 g_loss=1.101, d_r_acc=59, d_

Step31500, d_r_loss=0.653, d_f_loss=0.664 g_loss=0.783, d_r_acc=67, d_f_acc=63
Step31800, d_r_loss=0.680, d_f_loss=0.713 g_loss=0.790, d_r_acc=51, d_f_acc=50
Step32100, d_r_loss=0.718, d_f_loss=0.677 g_loss=0.752, d_r_acc=46, d_f_acc=62
Step32400, d_r_loss=0.715, d_f_loss=0.676 g_loss=0.786, d_r_acc=38, d_f_acc=57
Step32700, d_r_loss=0.693, d_f_loss=0.678 g_loss=0.771, d_r_acc=54, d_f_acc=59
Step33000, d_r_loss=0.657, d_f_loss=0.694 g_loss=0.759, d_r_acc=57, d_f_acc=52
Step33300, d_r_loss=0.692, d_f_loss=0.669 g_loss=0.757, d_r_acc=44, d_f_acc=64
Step33600, d_r_loss=0.672, d_f_loss=0.686 g_loss=0.763, d_r_acc=53, d_f_acc=58
Step33900, d_r_loss=0.656, d_f_loss=0.620 g_loss=0.796, d_r_acc=61, d_f_acc=75
Step34200, d_r_loss=0.658, d_f_loss=0.652 g_loss=0.777, d_r_acc=60, d_f_acc=60
Step34500, d_r_loss=0.628, d_f_loss=0.686 g_loss=0.774, d_r_acc=67, d_f_acc=59
Step34800, d_r_loss=0.650, d_f_loss=0.698 g_loss=0.768, d_r_acc=63, d_f_acc=50
Step35100, d_r_loss=0.724, d_f_loss=0.686 g_loss=0.7

## With the above metrics, we can see that the generator gets increasingly better at "tricking" the discriminator into misclassifying images due to the fake/generated images become more and more realistic. Probably an excessive number of epochs as the accuracy and loss metrics seem to level out about half way through the training

In [5]:
anim_file = 'dcgan_new.gif'
imgs = []
dir_name = "C:/Users/Michael/results_new9/"
f = 0
with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = sorted( filter( os.path.isfile,glob.glob(dir_name + 'generated_plot_*') ), key=os.path.getmtime )
    for file in filenames:
        f +=1
        image = imageio.imread(file)
        imgs.append(image)
        writer.append_data(image)
    image = imageio.imread(file)
    writer.append_data(image)
    imgs.append(image)
imageio.mimsave("results_new9/gifGAN_new1.gif",imgs, fps=2)

In [ ]:
embed.embed_file(anim_file)